<a href="https://colab.research.google.com/github/leshaz44/Aliaksei/blob/main/Task_3/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import sqlite3

# Загрузка файла CSV
df = pd.read_csv('data_Salary2.csv')
# Проверка загрузки
try:
    df = pd.read_csv('data_Salary2.csv')  # Читаем CSV-файл

    if len(df) > 0:               # Проверяем наличие записей в таблице
        print("Файл CSV успешно загружен.")
    else:
        print("Файл пустой!")
except FileNotFoundError:
    print("Ошибка: Файл data_Salary2.csv не найден.")
except Exception as e:
    print(f"Произошла ошибка при чтении файла: {e}")


# Подключение к базе данных SQLite
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Создание таблицы
cursor.execute('''CREATE TABLE IF NOT EXISTS table_name (
  id INTEGER PRIMARY KEY,
  Age INTEGER TEXT,
  Gender TEXT,
  Education TEXT,
  Job TEXT,
  Experience FLOAT,
  Salary FLOAT)''')

# Импортируем данные из DataFrame в созданную таблицу
df.to_sql('table_name', con=conn, index=False, if_exists='append')

# Выбор определенных столбцов и фильтрация результатов
cursor.execute("SELECT Salary, Age FROM table_name WHERE age > 30 LIMIT 20")
result = cursor.fetchall()
for row in result:
    print(row)

# Закрытие соединения
conn.commit()
conn.close()

Файл CSV успешно загружен.
(90000.0, 32)
(150000.0, 45)
(60000.0, 36)
(200000.0, 52)
(120000.0, 42)
(80000.0, 31)
(110000.0, 38)
(140000.0, 48)
(65000.0, 35)
(130000.0, 40)
(125000.0, 44)
(90000.0, 33)
(115000.0, 39)
(180000.0, 51)
(80000.0, 34)
(190000.0, 47)
(60000.0, 36)
(140000.0, 41)
(110000.0, 37)
(140000.0, 43)
